In [1]:
import pandas as pd
import numpy as np
import numpy.linalg as la
from sklearn.model_selection import train_test_split
from sklearn.base import clone
import timeit
import math

In [2]:
def pca(F, X):
    n, d = X.shape
    mu = np.zeros((d, 1))
    Z = np.zeros((d, F))
    for i in range(d):
        mu[i] = (1. / n) * np.sum(X[:, [i]])
    X = X - mu.T
    U, s, Vt = la.svd(X, False)
    g = s[:F]
    for i in range(F):
        g[i] = 1. / g[i]
    W = Vt[:F]
    Z = np.dot(W.T, np.diag(g))
    return (mu, Z)

def pca_proj(X,mu,Z):
    n, d = X.shape
    X = X - mu.T
    return np.dot(X, Z)

In [3]:
def k_fold(k, model, f, X, y, error_type="mse"):
    n, d = X.shape
    z = np.zeros((k, 1))
    for i in range(k):
        T = list(range(int((i * n) / k), int((n * (i + 1) / k))))
        S = [j for j in range(n) if j not in T]
        curr_model = clone(model)

        training_mu, training_Z = pca(f, X[S])
        training_X = pca_proj(X[S], training_mu, training_Z)

        curr_model.fit(training_X, y[S])

        test_X = pca_proj(X[T], training_mu, training_Z)

        # y[T] will be len(T) by 1
        # X[T] will be len(T) by d
        if error_type == "mse":
            z[i] = (1. / len(T)) * np.sum((y[T] - curr_model.predict(test_X)) ** 2)
        elif error_type == "log_mse":
            z[i] = (1. / len(T)) * np.sum((np.log(y[T] + 1) - np.log(curr_model.predict(test_X) + 1)) ** 2)
    return z

In [4]:
def bootstrapping(B, model, f, X, y, error_type="mse"):
    n, d = X.shape
    z = np.zeros((B, 1))
    for i in range(B):
        u = np.random.choice(n, n, replace=True)
        S = np.unique(u)
        T = np.setdiff1d(np.arange(n), S, assume_unique=True)
        curr_model = clone(model)

        training_mu, training_Z = pca(f, X[u])
        training_X = pca_proj(X[u], training_mu, training_Z)

        curr_model.fit(training_X, y[u])

        test_X = pca_proj(X[T], training_mu, training_Z)

        # y[T] will be len(T) by 1
        # X[T] will be len(T) by d
        # theta_hat will be d by 1
        if error_type == "mse":
            z[i] = (1. / len(T)) * np.sum((y[T] - curr_model.predict(test_X)) ** 2)
        elif error_type == "log_mse":
            z[i] = (1. / len(T)) * np.sum((np.log(y[T] + 1) - np.log(curr_model.predict(test_X) + 1)) ** 2)
    return z

In [5]:
def evaluate_model(model, f, X, y, k=5, B=5):
    ########################KFOLD###################
    print('Evaluating K-fold with %d folds.' % k)
    start_time = timeit.default_timer()
    k_fold_z = k_fold(k, model, f, X, y, error_type="log_mse")
    elapsed = timeit.default_timer() - start_time
    
    k_fold_mse = np.mean(k_fold_z)
    print('K-fold Mean Squared log Error: ', k_fold_mse)
    
    k_fold_rmse = math.sqrt(k_fold_mse)
    print('K-fold Square Root Mean Squared log Error: ', k_fold_rmse)

    print("Time elapsed for k-fold: ", elapsed)
    
    print()
    print()
    ###################BOOTSTRAPPING################
    print('Evaluating bootstrapping with %d bootstraps.' % B)
    start_time = timeit.default_timer()
    bootstrapping_z = bootstrapping(B, model, f, X, y)
    elapsed = timeit.default_timer() - start_time
    
    bootstrapping_mse = np.mean(bootstrapping_z)
    print('Bootstrapping Mean Squared Error: ', bootstrapping_mse)
    
    bootstrapping_rmse = math.sqrt(bootstrapping_mse)
    print('Bootstrapping Square Root Mean Squared Error: ', bootstrapping_rmse)

    print("Time elapsed for bootstrapping: ", elapsed)
    
    return (k_fold_z, k_fold_mse, k_fold_rmse, bootstrapping_z, bootstrapping_mse, bootstrapping_rmse)

# Data Processing

In [52]:
data.append(data).index + 1

Int64Index([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
            ...
            1451, 1452, 1453, 1454, 1455, 1456, 1457, 1458, 1459, 1460],
           dtype='int64', length=2920)

In [6]:

data_train = pd.read_csv("train.csv", header=0)
data_test = pd.read_csv("train.csv", header=0)

print(data.shape)

X = data.iloc[:,:-1]
Y = data.iloc[:,-1:]

print(X.shape)
print(Y.shape)

(1460, 81)
(1460, 80)
(1460, 1)


In [7]:
# this just sums up how many nulls per feature and divides to find percentage of nulls per feature
# if over 50% null then print the feature
data_keys = X.keys()
for i, b in enumerate((X.isnull().sum() / X.shape[0]) > 0.5):
    if b:
        print(data_keys[i])

Alley
PoolQC
Fence
MiscFeature


In [ ]:
# data = data.drop(['Alley', 'MiscFeature', 'Fence', 'PoolQC'], axis=1)

In [ ]:
# # Replaces categorical value in Quality columns with numerical scale
# qualityCols = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC',
#               'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond']

# X[qualityCols].head()

# for col in qualityCols:
#     # NA is never used since all NA's got converted to NaN objects when pandas read in the csv
#     X[col] = X[col].map({'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po':1, 'NA': 0})

# X[qualityCols].head()

In [8]:
# categorical columns
catCols = set(list(X))-set(list(X._get_numeric_data()))
print(catCols)

# #TRY dropping all cat cols
# data = data.drop(columns=catCols)

{'PoolQC', 'Heating', 'LandSlope', 'SaleType', 'GarageQual', 'ExterCond', 'Utilities', 'MSZoning', 'ExterQual', 'BsmtFinType2', 'BsmtFinType1', 'Alley', 'Exterior2nd', 'GarageCond', 'BsmtQual', 'BldgType', 'Neighborhood', 'Exterior1st', 'RoofStyle', 'PavedDrive', 'BsmtExposure', 'Fence', 'MiscFeature', 'SaleCondition', 'Functional', 'Condition2', 'Foundation', 'GarageType', 'FireplaceQu', 'LandContour', 'BsmtCond', 'LotShape', 'HeatingQC', 'MasVnrType', 'KitchenQual', 'HouseStyle', 'GarageFinish', 'Street', 'RoofMatl', 'Electrical', 'CentralAir', 'LotConfig', 'Condition1'}


In [9]:
len(catCols)

43

In [10]:
#Perform one hot encoding on all categorical columns
frames = []
for col in catCols:
    oneHot_encoded = pd.get_dummies(X[col])
    oneHot_encoded = oneHot_encoded.add_prefix(col + '_is_')
    frames.append(oneHot_encoded)

X = X.drop(catCols, axis=1)

X = pd.concat(frames, axis=1)

In [11]:
X.keys()

Index(['PoolQC_is_Ex', 'PoolQC_is_Fa', 'PoolQC_is_Gd', 'Heating_is_Floor',
       'Heating_is_GasA', 'Heating_is_GasW', 'Heating_is_Grav',
       'Heating_is_OthW', 'Heating_is_Wall', 'LandSlope_is_Gtl',
       ...
       'LotConfig_is_Inside', 'Condition1_is_Artery', 'Condition1_is_Feedr',
       'Condition1_is_Norm', 'Condition1_is_PosA', 'Condition1_is_PosN',
       'Condition1_is_RRAe', 'Condition1_is_RRAn', 'Condition1_is_RRNe',
       'Condition1_is_RRNn'],
      dtype='object', length=252)

In [12]:
X.isnull().values.any()

False

In [ ]:
# 80:20 train test ratio
test_size = 0.2
# This function splits the training and target sets into random train and test subsets.
# X_train and X_test are subsets of the training data
# y_train and y_test are subsets the the target data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size)

# PCA Feature Selection

In [13]:
F = 50
f = 50

In [14]:
X_mu, X_Z = pca(F, X.values)
X_pca = pca_proj(X.values, X_mu, X_Z)

In [15]:
print(X_mu.shape)
print(X_Z.shape)
print(X_pca.shape)

(252, 1)
(252, 50)
(1460, 50)


In [16]:
X_train_mu, X_train_Z = pca(F, X_train.values)

NameError: name 'X_train' is not defined

In [ ]:
print(X_train_mu.shape)
print(X_train_Z.shape)

In [ ]:
X_train_pca = pca_proj(X_train.values, X_train_mu, X_train_Z)
print(X_train_pca.shape)

X_test_pca = pca_proj(X_test.values, X_train_mu, X_train_Z)
print(X_test_pca.shape)

# AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
adaBoost = AdaBoostRegressor()
k_z, k_mse, k_rmse, b_z, b_mse, b_rmse = evaluate_model(adaBoost, f, X.values, Y.values.ravel(), k=5, B=5)

adaBoost.fit(X_train_pca, y_train.values.ravel())
adaBoost.score(X_test_pca, y_test.values.ravel())

In [ ]:
#View Predicted values
predicted = adaBoost.predict(X_test_pca)
ada_pred = y_test.copy()
ada_pred['predicted'] = predicted
ada_pred.head()

# XGBoost Regressor

In [ ]:
#!pip3 install xgboost

In [ ]:
from xgboost import XGBRegressor
xgb = XGBRegressor(max_depth=3, learning_rate=0.2, booster='gbtree', n_estimators=70)
k_z, k_mse, k_rmse, b_z, b_mse, b_rmse = evaluate_model(xgb, f, X.values, Y.values.ravel(), k=5, B=5)

xgb.fit(X_train_pca, y_train)
xgb.score(X_test_pca, y_test.values.ravel())

In [ ]:
predicted = xgb.predict(X_test_pca)
xgb_pred = y_test.copy()
xgb_pred['predicted'] = predicted
xgb_pred.head()

# SVM (SVR)

In [ ]:
from sklearn import svm
svr_model = svm.SVR(kernel="poly", coef0=-3500, gamma="auto")
# coef0 only works with poly and sigmoid kernels
# it just puts that value instead of the column of 1's

# without it, this model breaks for some reason

k_z, k_mse, k_rmse, b_z, b_mse, b_rmse = evaluate_model(svr_model, f, X.values, Y.values.ravel(), k=5, B=5)

# epsilon, degree
svr_model.fit(X_train_pca, y_train.values.ravel())
svr_model.score(X_test_pca, y_test.values.ravel())

In [ ]:
svr_predicted = svr_model.predict(X_test_pca)
svr_pred = y_test.copy()
svr_pred["predicted"] = svr_predicted
svr_pred.head()

# XGBoost seems the best so let's use it to submit to Kaggle.

In [40]:
data_kaggle = pd.read_csv("test.csv", header=0)
print(data_kaggle.shape)

X_kaggle_test = data_kaggle

(1459, 80)


(1459, 80)

In [42]:
#Perform one hot encoding on all categorical columns
frames_kaggle = []
for col in catCols:
    oneHot_encoded_kaggle = pd.get_dummies(X_kaggle_test[col])
    oneHot_encoded_kaggle = oneHot_encoded_kaggle.add_prefix(col + '_is_')
    frames_kaggle.append(oneHot_encoded_kaggle)

X_kaggle_test = X_kaggle_test.drop(catCols, axis=1)

X_kaggle_test = pd.concat(frames_kaggle, axis=1)

In [ ]:
X_kaggle_test.isnull().values.any()

In [ ]:
X_kaggle_test_pca = pca_proj(X_kaggle_test.values, X_mu, X_Z)

In [ ]:
from xgboost import XGBRegressor
xgb = XGBRegressor(max_depth=3, learning_rate=0.2, booster='gbtree', n_estimators=70)
xgb.fit(X_pca, Y.values.ravel())

In [ ]:
predicted = xgb.predict(X_kaggle_test_pca)